In [40]:
# !pip install transformers[torch]
# !pip install sentence_transformers

In [1]:
from transformers import AutoTokenizer
from sentence_transformers import SentenceTransformer
import json
import sys
import torch
import requests

In [2]:
device = 'cpu'

distilroberta = SentenceTransformer('stsb-distilroberta-base-v2', device=device)
distilroberta.max_seq_length = 512
usenc = SentenceTransformer('distiluse-base-multilingual-cased-v1', device=device)

usenc.max_seq_length = 512

In [3]:
seq = "13 cases of measles, 2 new cases of measles were contracted from abroad Published Sunday, February 24, 2019 12:57PM PST  Last Updated Sunday, February 24, 2019 8:06PM PST  There are now 13 people with the measles, according to health officials in Vancouver. Vancouver Coastal Health said the two new cases are not related to the outbreak involving two French language schools. The health authority said the two people were infected while they were travelling, but has only specified that one of them went to the Philippines. A spokesperson with the Ministry of Health confirmed to CTV News the second person had also travelled to the island nation.  VCH said the first measles patient arrived at the YVR on Feb. 11 on Philippine Airlines flight PR 116 from Manila and had a one-night layover. The patient left Vancouver to Edmonton on Air Canada Flight AC236 on Feb. 12. \"The safety and security of our passengers, employees and community is our top priority. We recommend anyone who was on the flights or in the terminal at YVR on those days and may be experiencing symptoms to contact 811 to talk to a nurse,\" YVR spokesperson Chris Devauld said in an e-mail. The second measles patient is a resident of the Lower Mainland and was infectious when they went to a restaurant and mall in Richmond, according to the health authority. VCH identified several places where people may have been exposed to the highly contagious virus, including: YVR, international arrivals and customs area between 4 and 7 p.m. on Feb. 11 Shuttle bus from YVR to La Quinta Inn in Richmond between 5 to 7 p.m. on Feb. 11 Shuttle bus from YVR to La Quinta Inn in Richmond between 8 and 10:30 a.m. on Feb. 12 YVR, Air Canada domestic departures area between 9 a.m. and noon on Feb. 12 Little Onpin Restaurant in Richmond between 6 and 10 p.m. on Feb. 16 Toys R Us at Lansdowne Centre between 11 a.m. and 9 p.m. on Feb. 17 Toys R Us at Lansdowne Centre between 11 a.m. and 9 p.m. on Feb. 18 'We can do better'  Health officials said those who have never had the measles or did not have the two doses of the vaccine are at the highest risk of contracting the virus. They encourage people to get fully immunized, adding the shot is free for everyone. Health Minister Adrian Dix assures all British Columbians that there is enough vaccine to deal with the demand. \"Frankly, people shouldn't be getting measles in the 21st century in British Columbia,\" he said. \"While our healthcare system is doing very well, I think can we do better and we will do better.\" Health Minister Dix says BC is in communications with other provinces and Washington State about the measles outbreak. Says 12 cases is too many and again implores people to get vaccinated @CTVVancouverpic.twitter.com/c1NjsbpeNi Other cases linked to French schools  On Friday, Vancouver Coastal Health announced there was a total of 10 cases that were \"very clearly associated\" with the schools, Ecole Secondaire Jules-Verne and Ecole Rose-Des-Vents. Officials said 33 students at the two schools have been ordered to stay home because they haven't provided proof of measles vaccination.  The other three cases were contracted while travelling and one unrelated case was acquired locally, according to VCH. Health officials are reminding people they can get vaccinations by visiting their family doctor, a walk-in clinic or urgent primary care centre, or a pharmacy for adults or children over five years old. Calling ahead is also recommended."

In [4]:
distilroberta_embedding = distilroberta.encode(seq)
use_embedding = usenc.encode(seq)

In [5]:
# this was computed separately, contextualized topic modeling with 50 topics
topics_ctm_50 = [0.021997988456860185,
  0.008681272948160767,
  0.003812129539437592,
  0.015278012864291668,
  0.017179515771567823,
  0.005514769861474633,
  0.012595915887504815,
  0.005087137268856168,
  0.22096834182739258,
  0.030134920682758092,
  0.01751194945536554,
  0.020441827783361077,
  0.017262511944863945,
  0.007391538983210921,
  0.015647229086607695,
  0.010250990465283393,
  0.046074136369861664,
  0.014497667644172907,
  0.026647721976041795,
  0.00719403363764286,
  0.017288078647106887,
  0.03904122076928616,
  0.0027577191591262817,
  0.005326130893081427,
  0.012412398681044578,
  0.02106521129608154,
  0.02921512946486473,
  0.015480046533048153,
  0.010256040841341019,
  0.003950248612090946,
  0.05219747200608253,
  0.01412213551811874,
  0.00968298246152699,
  0.012711086496710777,
  0.02313218852505088,
  0.007741716736927628,
  0.010643875645473599,
  0.022799173183739186,
  0.01860669073648751,
  0.025112610310316086,
  0.0050189508590847256,
  0.023096046410501003,
  0.005882679286878556,
  0.042070050910115245,
  0.008823895640671254,
  0.007273900741711259,
  0.006751537532545626,
  0.00692114511039108,
  0.00793951191008091,
  0.008510466339066625]

In [6]:
alpha = 0.5
beta = 0.5

In [7]:
query_distilroberta = {"size":10, 
         "_source": "content_t",
         "query": {
            "script_score": {
              "query" : {
                "match_all" : {}
                },
              "script": {
                "lang" : "painless",
                "source": "if ((1 + cosineSimilarity(params.queryTopic, 'topics_ctm_50') >= 0) && (1 + cosineSimilarity(params.queryTopic, 'topics_ctm_50') <= 2)) {params.alpha * (1.0+cosineSimilarity(params.queryVector, 'distilroberta_embedding')) + params.beta * (1.0+cosineSimilarity(params.queryTopic, 'topics_ctm_50'))} else {params.alpha * (1.0+cosineSimilarity(params.queryVector, 'distilroberta_embedding'))}",
                "params": {
                    "queryVector": distilroberta_embedding.tolist(),
                    "queryTopic": topics_ctm_50,
                    "alpha" : alpha,
                    "beta" : beta
                    }
                }
            }
          }
        }

In [8]:
response = requests.get('http://localhost:9200/posts/_search', json=query_distilroberta)
print(json.dumps(response.json(), indent=1))

{
 "took": 158453,
 "timed_out": false,
 "_shards": {
  "total": 1,
  "successful": 1,
  "skipped": 0,
  "failed": 0
 },
 "hits": {
  "total": {
   "value": 10000,
   "relation": "gte"
  },
  "max_score": 1.9748241,
  "hits": [
   {
    "_index": "posts",
    "_type": "_doc",
    "_id": "43516898814867403655109610208249355561",
    "_score": 1.9748241,
    "_source": {
     "content_t": "13 cases of measles, 2 new cases of measles were contracted from abroad Published Sunday, February 24, 2019 12:57PM PST  Last Updated Sunday, February 24, 2019 8:06PM PST  There are now 13 people with the measles, according to health officials in Vancouver. Vancouver Coastal Health said the two new cases are not related to the outbreak involving two French language schools. The health authority said the two people were infected while they were travelling, but has only specified that one of them went to the Philippines. A spokesperson with the Ministry of Health confirmed to CTV News the second person h